In [1]:
## uncomment for the first run
# import Pkg
# Pkg.add("PyCall")
# Pkg.add("Conda")
# ENV["PYTHON"] = "/opt/anaconda3/envs/turi/bin/python"
# /opt/anaconda3/envs/turi/lib/python3.6/site-packages/
# Pkg.build("PyCall")

In [2]:
using PyCall
tc = pyimport("turicreate")

PyObject <module 'turicreate' from '/opt/anaconda3/envs/turi/lib/python3.6/site-packages/turicreate/__init__.py'>

In [3]:
data_path="./DATA/processed/uber.csv"
data = tc.SFrame(data_path)
println("data is a ", typeof(data))

Finished parsing file /Users/uki/REPOS/MarketIndicators.jl/DATA/processed/uber.csv
Parsing completed. Parsed 100 lines in 0.040542 secs.
------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
Finished parsing file /Users/uki/REPOS/MarketIndicators.jl/DATA/processed/uber.csv
Parsing completed. Parsed 1744 lines in 0.008598 secs.
data is a PyObject


In [4]:
row_count = size(data)[1]

1744

In [11]:
# Do not take initial year data as averages are not complete
#data[1:1]

In [6]:
# Make a train-test split
train_data, test_data = data.random_split(0.8)
println()

In [7]:
size(train_data)
size(test_data)

(347,)

In [8]:
column_to_predict = "DJIA_Original"

model = tc.regression.create( 
    train_data, 
    target = column_to_predict, 
    features = [
                 "DJIA_Avg005"
                ,"DJIA_Avg030"
                ,"DJIA_Avg090"
                ,"DJIA_Avg180"        
                ,"DJIA_Avg365"

                ,"ISM_MFC_EMP_Avg090"
                ,"ISM_MFC_EMP_Avg180"
                ,"ISM_MFC_EMP_Avg365"

                ,"HOUSE_SRT_MM_Value"
                ,"HOUSE_SRT_MM_Avg090"
                ,"HOUSE_SRT_MM_Avg180"
                ,"HOUSE_SRT_MM_Avg365"

                ,"MFC_MPI_Value"
                ,"MFC_MPI_Avg090"
                ,"MFC_MPI_Avg180"
                ,"MFC_MPI_Avg365"
            ], 
    validation_set="auto", 
    verbose=true
)

Automatically generating validation set from 5% of the data.
Linear regression:
--------------------------------------------------------
Number of examples          : 1326
Number of features          : 16
Number of unpacked features : 16
Number of coefficients    : 17
Starting Newton Method
--------------------------------------------------------
+-----------+----------+--------------+--------------------+----------------------+---------------------------------+-----------------------------------+
| Iteration | Passes   | Elapsed Time | Training Max Error | Validation Max Error | Training Root-Mean-Square Error | Validation Root-Mean-Square Error |
+-----------+----------+--------------+--------------------+----------------------+---------------------------------+-----------------------------------+
| 1         | 2        | 1.014473     | 1776.716312        | 766.914945           | 214.153236                      | 177.275959                        |
+-----------+----------+-----------

PyObject Class                          : LinearRegression

Schema
------
Number of coefficients         : 17
Number of examples             : 1326
Number of feature columns      : 16
Number of unpacked features    : 16

Hyperparameters
---------------
L1 penalty                     : 0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 1.0214

Settings
--------
Residual sum of squares        : 60812492.8212
Training RMSE                  : 214.1532

Highest Positive Coefficients
-----------------------------
(intercept)                    : 15717.79
DJIA_Avg005                    : 125.6601
DJIA_Avg180                    : 4.6502
MFC_MPI_Avg365                 : 3.5494
ISM_MFC_EMP_Avg180             : 3.0015

Lowest Negative Coefficients
----------------------------
DJIA_Avg030               

In [9]:
## Save predictions to an SArray
predictions = model.predict(test_data)
#predictions

PyObject dtype: float
Rows: 347
[16380.452477226856, 16335.20998699159, 16121.735165103286, 16129.609848280343, 16347.19052416887, 16512.765114142796, 16519.607394058054, 16080.939168706764, 16060.532255620641, 16158.286831686939, 16380.05495440027, 16478.36453751449, 16739.98279427749, 16785.175819549277, 17024.07428161827, 17235.669238467155, 17298.847671984353, 17744.29896417558, 17735.6807022313, 17844.2233663157, 17705.334507076735, 17797.617663370653, 17917.147450858112, 17965.34032828885, 17989.547363071484, 18045.77176640772, 17843.139328752965, 17817.403542247208, 17814.491687654197, 17578.624479546037, 17910.69595822855, 17860.537563672806, 17835.316706632384, 17834.500144000816, 17932.529856181984, 18007.272759600637, 17977.57385607022, 18348.887910171026, 18607.79189653252, 18468.32733977436, 18565.70409585579, 18586.55385711259, 18563.220928058046, 18537.072710098033, 18221.958001286253, 18175.836003920896, 18222.107031743362, 18327.72767282045, 18331.85833162359, 18287.62

In [34]:
initio = 0                    # latin: start
gradus = 30                   # latin: step
finem  = size(predictions)[1] # latin: end

println(column_to_predict, " ", finem)

for id in initio:gradus:finem
    
    a = get(predictions, id)
    a = round(a, digits=2)
    
    b = get(test_data, column_to_predict )
    b = get(b, id )
    b = round(b, digits=2)

    println( "predicted ", a, "\t, but actual value was \t", b , "\t difference is ",  round(b-a, digits=2)  ) # dict
end

DJIA_Original 347
predicted 16380.45	, but actual value was 	16354.33	 difference is -26.12
predicted 17910.7	, but actual value was 	17809.18	 difference is -101.52
predicted 18465.69	, but actual value was 	18873.66	 difference is 407.97
predicted 20792.24	, but actual value was 	20718.33	 difference is -73.91
predicted 22168.06	, but actual value was 	22057.19	 difference is -110.87
predicted 25289.04	, but actual value was 	25311.99	 difference is 22.95
predicted 24842.65	, but actual value was 	24635.18	 difference is -207.47
predicted 25120.7	, but actual value was 	24916.16	 difference is -204.54
predicted 25928.78	, but actual value was 	25645.45	 difference is -283.33
predicted 27254.26	, but actual value was 	27161.93	 difference is -92.33
predicted 23359.06	, but actual value was 	23189.76	 difference is -169.3
predicted 28183.36	, but actual value was 	28400.74	 difference is 217.38


In [35]:
#TODO: write this in a loop to select the best model
# Evaluate the model and save the results into a dictionary
results = model.evaluate( test_data ) #test_data[0:2531]
results

Dict{Any,Any} with 2 entries:
  "rmse"      => 225.762
  "max_error" => 1847.16

In [51]:
rmse = results["rmse"]
rmse = round(rmse, digits=2)

max_error = results["max_error"]
max_error = round(max_error, digits=2)

println( "max_error: ", max_error, ", rmse: ", rmse )

max_error: 1847.16, rmse: 225.76
